In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
test_df.head()

In [ ]:
submission_df = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
submission_df.head()

In [ ]:
submission_df['count'].value_counts()

In [ ]:
AllData=[train_df,test_df]

In [ ]:
print(train_df.isnull().sum())
print('*'*50)
print(test_df.isnull().sum())

In [ ]:
train_df.describe()

In [ ]:
for dataset in AllData:  
    dataset['datetime'] =  pd.to_datetime(dataset['datetime'], format="%Y-%m-%d %H:%M:%S")
    dataset['year'] = dataset['datetime'].dt.year
    dataset['month'] = dataset['datetime'].dt.month
    dataset['hour'] = dataset['datetime'].dt.hour
    dataset['weekday'] = dataset['datetime'].dt.dayofweek


In [ ]:
train_df.info()

In [ ]:
    
categoryColumns=["hour","weekday","month","season","weather","holiday","workingday"]
for var in categoryColumns:
    train_df[var] = train_df[var].astype("category")
    test_df[var] = test_df[var].astype("category")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sns.boxplot(data=train_df,y="count",orient="v",ax=axes[0][0])
sns.boxplot(data=train_df,y="count",x="season",orient="v",ax=axes[0][1])
sns.boxplot(data=train_df,y="count",x="hour",orient="v",ax=axes[1][0])
sns.boxplot(data=train_df,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Box Plot On Count")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Box Plot On Count Across Season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Box Plot On Count Across Working Day")

In [ ]:
print('shape with outliers: ',train_df.shape)
train_df = train_df[np.abs(train_df["count"]-train_df["count"].mean())<=(3*train_df["count"].std())]
print('shape without outliers: ',train_df.shape)

In [ ]:
train_df.columns

In [ ]:
train_df  = train_df.drop(["datetime"],axis=1)
test_df  = test_df.drop(["datetime"],axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))         
sns.heatmap(train_df.corr(), annot = True,ax=ax)

In [ ]:
train_df  = train_df.drop(["atemp"],axis=1)
test_df  = test_df.drop(["atemp"],axis=1)

train_df['year'].value_counts()

In [ ]:
train_df['year'] = train_df['year'].map({2011:1,2012:2}).astype('category')
test_df['year'] = test_df['year'].map({2011:1,2012:2}).astype('category')
train_df  = train_df.drop(['casual', 'registered'],axis=1)
train_df

In [ ]:
test_df

In [ ]:
X=train_df.drop(columns=['count'])
y=train_df['count']
x_test=test_df

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
from sklearn.metrics import mean_squared_log_error
Lr= LinearRegression()
Lr.fit(x_train,y_train)
y_pred= Lr.predict(x_val)
print(np.sqrt(mean_squared_log_error(abs(y_val), abs(y_pred))))

In [ ]:
from math import sqrt
def RMSLE(y_pred , y_actual):
    n = y_pred.size 
    RMSLE = sqrt(((np.log(y_pred+1)-np.log(y_actual+1))**2).sum()/n)
    return RMSLE

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from numpy import arange

model = Ridge()
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error',  n_jobs=-1)
results = search.fit(x_train, y_train)

print('Config: %s' % results.best_params_)

In [ ]:
# Create the Ridge model using best alpha value:
from sklearn.linear_model import Ridge
rid_reg = Ridge(alpha=0.9)
rid_reg.fit(x_train, y_train)

Y_pred_ridge = rid_reg.predict(x_train)
Y_pred_ridge_val = rid_reg.predict(x_val)

R_ridge = RMSLE(Y_pred_ridge_val , y_val)
R1_ridge = RMSLE(Y_pred_ridge , y_train)

print("RMSLE of  Ridge : ",R_ridge)
print('RMSLE of of what model learn Ridge: ',R1_ridge)

In [ ]:
# Create Lasso model
from sklearn.linear_model import Lasso
ls = Lasso(alpha = 0.8)
ls.fit(x_train, y_train)

Y_pred_ls = ls.predict(x_train)
Y_pred_ls_val = ls.predict(x_val)

R_ls = RMSLE(Y_pred_ls_val , y_val)
R1_ls = RMSLE(Y_pred_ls , y_train)

print("RMSLE of  Ridge : ",R_ls)
print('RMSLE of of what model learn Ridge: ',R1_ls)

In [ ]:
from sklearn.ensemble import RandomForestRegressor


Regressor = RandomForestRegressor( max_depth=None, random_state=0)


Regressor = Regressor.fit(x_train, y_train)
y_pred_RF = Regressor.predict(x_val)
y_1 = Regressor.predict(x_train)


R = RMSLE(y_pred_RF , y_val)
R1 = RMSLE(y_1 , y_train)
print("RMSLE of Random Forest Regressor :",R)
print("RMSLE of what model learn :",R1)

In [ ]:
y_test_predicted = Regressor.predict(x_test)


In [ ]:
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")


submission_df = pd.DataFrame({
        "datetime": test["datetime"],
        "count": y_test_predicted
    })

In [ ]:
submission_df.to_csv("submission.csv", index = False)

In [ ]:
submission_df